In [1]:
import pymysql
import pymysql.cursors

In [2]:
conn = pymysql.connect(host='localhost', user = 'root', password='postech', db = 'kowiki', cursorclass=pymysql.cursors.DictCursor)
cursor = conn.cursor()

In [3]:
sql = 'SELECT *'
sql += 'FROM categorylinks A, page B '
sql += 'WHERE A.cl_from = B.page_id'
cursor.execute(sql)
category_link_data = cursor.fetchall()

In [4]:
len(category_link_data)

4382534

In [5]:
import pandas as pd
joined_data = pd.DataFrame(category_link_data, columns = ['cl_from', 'page_title', 'page_namespace', 'page_len','cl_type','cl_to'])
joined_data = joined_data[joined_data.page_title != joined_data.cl_to]

In [6]:
len(joined_data)

4345131

In [7]:
joined_data['page_title'] = joined_data['page_title'].str.decode("utf-8")
joined_data['cl_to'] = joined_data['cl_to'].str.decode("utf-8")

In [8]:
import numpy as np
joined_data['edge'] = list(zip(joined_data.page_title, joined_data.cl_to))
real_data = joined_data[(joined_data.page_namespace == 14)|(joined_data.page_namespace == 0)]

In [9]:
len(real_data)

3469785

In [10]:
root_node = '과학기술'

In [11]:
delete_list = list(real_data[real_data.page_title==root_node]['cl_to'])

In [12]:
delete_list

['글로벌세계대백과를_인용한_문서', '영어_표기를_포함한_문서', '주제_분류']

In [13]:
real_data = real_data[real_data.page_title!=root_node]

In [14]:
len(real_data)

3469782

In [15]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(real_data.edge)

In [19]:
def size_of_networks(graph):
    return {'number_of_nodes': len(graph.nodes()), 'number_of_links': len(graph.edges()), 'ratio': len(graph.edges())/len(graph.nodes())}

In [20]:
size_of_networks(G)

{'number_of_links': 3441157,
 'number_of_nodes': 663564,
 'ratio': 5.185870541500142}

In [21]:
node_set = list(nx.ancestors(G, root_node)) + [root_node]

In [22]:
len(nx.ancestors(G, root_node))

646047

In [23]:
len(nx.descendants(G, root_node))

0

In [25]:
subgraph = nx.subgraph(G, node_set)

In [26]:
nx.write_gml(subgraph, 'network_data/kowiki.gml')
pd.Series(subgraph.nodes()).to_pickle('network_data/kowiki.pkl')

In [27]:
size_of_networks(subgraph)

{'number_of_links': 3421520,
 'number_of_nodes': 646048,
 'ratio': 5.296077071672693}